In [6]:
!pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 25.4 MB/s eta 0:00:0000:0100:01


In [2]:
!python snake_game_human.py --block_size 20 --speed 10 --width 800 --height 600

Traceback (most recent call last):
  File "/Users/carlosperezricardo/Documents/GitHub/Reinforcement-Learning-Snake/snake_game_human.py", line 1, in <module>
    import pygame
ModuleNotFoundError: No module named 'pygame'


# AI GAME

- Init reward: 0
- Reward per collision: -10
- Reward per divagar / no anar a pel premi: -10
    - Ambdos casos son GAME OVER
- Reward per menjar: +10

In [3]:
# game.py 
BLOCK_SIZE = 20
SPEED = 40

def play_step(self, action):
        self.frame_iteration += 1

        # 3. check if game over
        reward = 0
        
        game_over = False
        if self.is_collision() or self.frame_iteration > 100*len(self.snake):
            game_over = True
            reward = -10
            return reward, game_over, self.score

        # 4. place new food or just move
        if self.head == self.food:
            self.score += 1
            reward = 10
            self._place_food()
        else:
            self.snake.pop()
        
        # 6. return game over and score
        return reward, game_over, self.score

# AGENT

## STATES

Booleans que indiquen si:

1. Danger straight
2. Danger right
3. Danger left
3. dir left
4. dir right
5. dir up
6. dir down
7. el menjar està a l'esquerra: game.food.x < game.head.x,  # food left
8. el menjar està a la dreta: game.food.x > game.head.x,  # food right
9. el menjar està a sota: game.food.y < game.head.y,  # food up
10. el menjar està a sobre: game.food.y > game.head.y  # food down

In [2]:
#!python agent.py
!python agent.py --block_size 20 --speed 50 --width 200 --height 200 --discount 0.9 --max_memory 100000 --batch_size 1000 --learning_rate 0.001

pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
/Users/carlosperezricardo/Documents/GitHub/Reinforcement-Learning-Snake/model.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  state = torch.tensor(state, dtype=torch.float)
Game 1 Score 0 Record: 0
Figure(640x480)
Figure(640x480)
Game 2 Score 0 Record: 0
Figure(640x480)
Figure(640x480)
Game 3 Score 0 Record: 0
Figure(640x480)
Figure(640x480)
Game 4 Score 1 Record: 1
Figure(640x480)
Figure(640x480)
Game 5 Score 1 Record: 1
Figure(640x480)
Figure(640x480)
Game 6 Score 0 Record: 1
Figure(640x480)
Figure(640x480)
Game 7 Score 0 Record: 1
Figure(640x480)
Figure(640x480)
Game 8 Score 0 Record: 1
Figure(640x480)
Figure(640x480)

In [5]:
import torch
import random
import numpy as np
from collections import deque
from game import SnakeGameAI, Direction, Point
from model import Linear_QNet, QTrainer
from helper import plot

MAX_MEMORY = 100_000
BATCH_SIZE = 1000
LR = 0.001

class Agent:

    def __init__(self):
        self.n_games = 0
        self.epsilon = 0 # randomness
        self.gamma = 0.9 # discount rate
        self.memory = deque(maxlen=MAX_MEMORY) # popleft()
        self.model = Linear_QNet(11, 256, 3)
        self.trainer = QTrainer(self.model, lr=LR, gamma=self.gamma)


    def get_state(self, game):
        block_size = 20
        
        head = game.snake[0]
        point_l = Point(head.x - block_size, head.y)
        point_r = Point(head.x + block_size, head.y)
        point_u = Point(head.x, head.y - block_size)
        point_d = Point(head.x, head.y + block_size)
        
        dir_l = game.direction == Direction.LEFT
        dir_r = game.direction == Direction.RIGHT
        dir_u = game.direction == Direction.UP
        dir_d = game.direction == Direction.DOWN
        
        state = [
            # Danger straight
            (dir_r and game.is_collision(point_r)) or 
            (dir_l and game.is_collision(point_l)) or 
            (dir_u and game.is_collision(point_u)) or 
            (dir_d and game.is_collision(point_d)),

            # Danger right
            (dir_u and game.is_collision(point_r)) or 
            (dir_d and game.is_collision(point_l)) or 
            (dir_l and game.is_collision(point_u)) or 
            (dir_r and game.is_collision(point_d)),

            # Danger left
            (dir_d and game.is_collision(point_r)) or 
            (dir_u and game.is_collision(point_l)) or 
            (dir_r and game.is_collision(point_u)) or 
            (dir_l and game.is_collision(point_d)),
            
            # Move direction
            dir_l,
            dir_r,
            dir_u,
            dir_d,
            
            # Food location 
            game.food.x < game.head.x,  # food left
            game.food.x > game.head.x,  # food right
            game.food.y < game.head.y,  # food up
            game.food.y > game.head.y  # food down
            ]
        
        return np.array(state, dtype=int)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # popleft if MAX_MEMORY is reached

    def train_long_memory(self):
        if len(self.memory) > BATCH_SIZE:
            mini_sample = random.sample(self.memory, BATCH_SIZE) # list of tuples
        else:
            mini_sample = self.memory

        states, actions, rewards, next_states, dones = zip(*mini_sample)
        self.trainer.train_step(states, actions, rewards, next_states, dones)
        #for state, action, reward, nexrt_state, done in mini_sample:
        #    self.trainer.train_step(state, action, reward, next_state, done)

    def train_short_memory(self, state, action, reward, next_state, done):
        self.trainer.train_step(state, action, reward, next_state, done)

    def get_action(self, state):
        # random moves: tradeoff exploration / exploitation
        self.epsilon = 80 - self.n_games
        final_move = [0,0,0]
        if random.randint(0, 200) < self.epsilon:
            move = random.randint(0, 2)
            final_move[move] = 1
        else:
            state0 = torch.tensor(state, dtype=torch.float)
            prediction = self.model(state0)
            move = torch.argmax(prediction).item()
            final_move[move] = 1

        return final_move


def train():
    plot_scores = []
    plot_mean_scores = []
    total_score = 0
    record = 0
    agent = Agent()
    game = SnakeGameAI()
    while True:
        # get old state
        state_old = agent.get_state(game)

        # get move
        final_move = agent.get_action(state_old)

        # perform move and get new state
        reward, done, score = game.play_step(final_move)
        state_new = agent.get_state(game)

        # train short memory
        agent.train_short_memory(state_old, final_move, reward, state_new, done)

        # remember
        agent.remember(state_old, final_move, reward, state_new, done)

        if done:
            # train long memory, plot result
            game.reset()
            agent.n_games += 1
            agent.train_long_memory()

            if score > record:
                record = score
                agent.model.save()

            print('Game', agent.n_games, 'Score', score, 'Record:', record)

            plot_scores.append(score)
            total_score += score
            mean_score = total_score / agent.n_games
            plot_mean_scores.append(mean_score)
            plot(plot_scores, plot_mean_scores)


if __name__ == '__main__':
    train()

ModuleNotFoundError: No module named 'pygame'